# Quantum Fourier Transform

Without getting into maths details, QFT algorithm of n qbits is made by applying Hadamard gate on each qbit with controlled phase shifts in between. The amount of phase shift is $\pi,\pi/2,\pi/4,\pi/8$ etc depending on "distance" between qbits.

As concrete example 4qbit QFT is depicted as 
![QFT circuit](images/QFT400.png "Quantum Fourier Transform of 4 Qbits")

Let us make that using gates we have

In [1]:
from qcomp.qbit import *
from qcomp.qregister import mk_reg
from qcomp.gate import VChain, HChain, CPSGate, H, I

We observe on every step there are bits that nothing is applied to. These are 'spectator bits'. Actually there is gate being applied to them, Identity gate. Firstly Hadamard gates in the diagram are constructed as

In [2]:
H0 = VChain([H] + [I]*3)
H1 = VChain([I] + [H] + [I]*2)
H2 = VChain([I]*2 + [H] + [I])
H3 = VChain([I]*3 + [H])

Phase gates are a bit tricky to make as they need two qbits which may not be neighbours. So we also need to specify which bits they act on. I also use naming convention where three indices nmk are used. Meaning is, this gate is kth gate between nth and mth Hadamard. Irritating things is bottom bits are control and top one is acting bit which is usually opposite in diagrams.

In [3]:
# remember we can always do this when not sure how things work
help(CPSGate)

Help on class CPSGate in module qcomp.gate:

class CPSGate(CGate)
 |  Controlled Phase Shift Gate
 |  
 |  Method resolution order:
 |      CPSGate
 |      CGate
 |      SGate
 |      Gate
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, phi, cbit=0, abit=1, qreg_size=2)
 |      Parameters
 |      phi: amount of shift
 |      cbit: control bit
 |      abit: bit that will it act on
 |      qreg_size: size of Quantum register
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from SGate:
 |  
 |  apply(self, qreg)
 |      Apply the gate to the given register 
 |      Parameters
 |      -----------
 |      **qreg** : QReg instance 
 |      
 |      Returns
 |      ----------
 |      new_qreg : QReg object created after transformation
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from Gate:
 |  
 |  __dict__
 |      dictionary for instance variab

In [4]:
cps010 = CPSGate(np.pi,   1, 0,4)
cps120 = CPSGate(np.pi/2, 2, 0,4)
cps121 = CPSGate(np.pi,   2, 1,4)
cps230 = CPSGate(np.pi/4, 3, 0,4)
cps231 = CPSGate(np.pi/2, 3, 1,4)
cps232 = CPSGate(np.pi,   3, 2,4)

Now we can complete the diagram

In [5]:
qft = HChain([H0, cps010, H1, cps120, cps121, H2, cps230, cps231, cps232, H])

Now we can prepare any quantum register (4bit) we want and apply QFT to it

In [6]:
Q0000 = mk_reg([ZERO]*4)

In [7]:
qft.apply(Q0000)

0.43+0.00j 	 |0000> 
-0.09-0.00j 	 |0001> 
0.17+0.00j 	 |0010> 
-0.17-0.00j 	 |0011> 
0.34+0.09j 	 |0100> 
-0.17-0.09j 	 |0101> 
0.09+0.17j 	 |0110> 
-0.26-0.17j 	 |0111> 
0.26+0.00j 	 |1000> 
-0.26-0.00j 	 |1001> 
0.17+0.00j 	 |1010> 
-0.17-0.00j 	 |1011> 
0.26+0.09j 	 |1100> 
-0.26-0.09j 	 |1101> 
0.17+0.17j 	 |1110> 
-0.17-0.17j 	 |1111> 

# Accuracy check

QFT is explained well in https://www.robots.ox.ac.uk/~parg/pubs/qcf.pdf. I ll try to replicate that work. Only change is 3bit system is utilized.

In [56]:
H0 = VChain([H] + [I]*2)
H1 = VChain([I] + [H] + [I])
H2 = VChain([I]*2 + [H])

cps010 = CPSGate(np.pi,   1, 0,3)
cps120 = CPSGate(np.pi/2, 2, 0,3)
cps121 = CPSGate(np.pi,   2, 1,3)

qft3 = HChain([H0, cps010, cps120, H1, cps121, H2])

In [57]:
from qcomp.gate import CNOT, NOT
# psi has two states 101 and 011, these states are entangled (0th and 1s qbit)
# so we have to use two bit gates
psi = mk_reg([ZERO,ZERO,ONE])
psi = HChain([ VChain([H,I,I]), VChain([NOT,I,I]), CNOT(0,1,3), VChain([NOT,I,I]) ]).apply(psi)
psi

0.00+0.00j 	 |000> 
0.00+0.00j 	 |001> 
0.00+0.00j 	 |010> 
0.71+0.00j 	 |011> 
0.00+0.00j 	 |100> 
0.71+0.00j 	 |101> 
0.00+0.00j 	 |110> 
0.00+0.00j 	 |111> 

In [58]:
qft3.apply(psi)

0.00+0.00j 	 |000> 
0.00+0.00j 	 |001> 
0.00+0.00j 	 |010> 
0.00+0.00j 	 |011> 
-0.00-0.15j 	 |100> 
0.00+0.15j 	 |101> 
0.31-0.62j 	 |110> 
0.31+0.62j 	 |111> 